# unet2D tutorial notebook

This notebook is part of the unet2D tutorial. Please refer to t03_unet2D.pdf for the full tutorial.

---
The following code cell generates 400x400 images for the unet2D tutorial. These are similar to the images generated for the pixelclassifier tutorial (see t01_pixelclassifier.ipynb).

In [ ]:
# -----
# TRAIN/TEST GENERATION CELL
# -----

import numpy as np
import matplotlib.pyplot as plt
from gpfunctions import generateSynthCellsImage, imshow, imshowlist, \
    imerode, imdilate, tifwrite, im2double, tifread, createFolderIfNonExistent
%matplotlib inline

train_folder = 'DataForUNet2D/Train'
test_folder = 'DataForUNet2D/Test'
createFolderIfNonExistent(train_folder)
createFolderIfNonExistent(test_folder)

def gen_img_lbl_pair():
    I, L = generateSynthCellsImage(im_size=400, n_circs=20)

    cells = imdilate(L > 0, 1)
    interior = imerode(cells, 2)
    contours = np.logical_and(cells, np.logical_not(interior))
    not_contours = np.logical_not(contours)

    c_cont = np.sum(contours)

    f = c_cont/np.sum(not_contours)
    not_contours = np.logical_and(not_contours, np.random.rand(I.shape[0], I.shape[1]) < f)
    
    print('n pixels class 0:', np.sum(contours), '| npixels class 1:', np.sum(not_contours))

    L[:] = 0
    L[contours] = 1
    L[not_contours] = 2
    
    return I, L
    
for idx in range(20):
    print('train image', idx)
    
    I, L = gen_img_lbl_pair()
    tifwrite(np.uint8(255*I), '%s/I%05d_Img.tif' % (train_folder, idx))
    tifwrite(L, '%s/I%05d_Ant.tif' % (train_folder, idx))

for idx in range(5):
    print('test image', idx)
    
    I, L = gen_img_lbl_pair()
    tifwrite(np.uint8(255*I), '%s/I%05d_Img.tif' % (test_folder, idx))
    tifwrite(L, '%s/I%05d_Ant.tif' % (test_folder, idx))

---
The following code cell splits the images of the training set above into tiles of 60x60. unet2D.py expects the training set to contain [image, label] pairs of this size.

In [ ]:
# -----
# TILING CELL
# -----

from gpfunctions import splitIntoTiles2D

tiling_folder = 'DataForUNet2D/Train_60'
createFolderIfNonExistent(tiling_folder)

splitIntoTiles2D(train_folder, tiling_folder)

---
The following code cell generates some images for unet2D.py in 'deployment' mode.

In [ ]:
# -----
# DEPLOY GENERATION CELL
# -----

deploy_folder_in = 'DataForUNet2D/Deploy_In'
deploy_folder_out = 'DataForUNet2D/Deploy_Out'
createFolderIfNonExistent(deploy_folder_in)
createFolderIfNonExistent(deploy_folder_out)

for idx in range(5):
    print('deploy image', idx)
    
    I, _ = gen_img_lbl_pair()
    tifwrite(np.uint8(255*I), '%s/I%05d_Img.tif' % (deploy_folder_in, idx))